# **Defining the inputs**

In [2]:
import pandas as pd

data = pd.DataFrame({'1' : [1, 2, 3], '2' : [2, 2, 5], '3' : [3, 3, 3],
                   '4' : [4, 2, 6], '5' : [5, 3, 3], '6' : [6, 6, 2],
                   '7' : [ 7, 3, 4], '8' : [8, 1, 8], '9' : [9, 4, 7],
                   '10' : [10, 4, 4], '11' : [11, 11, 12]}, index = ['Job', 'Pj', 'dj']).T

# **Defining the functions**

In [3]:
def SPT_EDD(df):
    # Sort jobs by processing time in ascending order
    return df.sort_values(by = ['Pj', 'dj'])

In [4]:
def EDD(df):
  return df.sort_values(by = ['dj'])

In [5]:
def Lmax_EDD(n, EDD):
    Lj = []
    total_process_time = 0
    for i in range(n):
        total_process_time += EDD['Pj'][i]
        if total_process_time > EDD['dj'][i]:
            Lj.append(total_process_time - EDD['dj'][i])
    return max(Lj)

# **Trade-Offs between Total Completion Time and Maximum Lateness**

In [6]:
# Define lists to store the points for the trade-off curve
total_completion_times = []
maximum_latenesses = []

# Step 1
r = 1
EDD = EDD(data)
SPT_EDD = SPT_EDD(data)
n = EDD.shape[0]
Lmax = Lmax_EDD(n, EDD)
EDD['dd'] = EDD['dj'] + Lmax
Lmax_SPT_EDD = Lmax_EDD(n, SPT_EDD)

# Step 2
while Lmax <= Lmax_SPT_EDD:
    sequence = []
    k = n
    Jc = list(EDD['Job'])
    tau = EDD['Pj'].sum()
    delta = tau

    # Step 3
    while k >= 1:
        L = EDD.loc[(EDD['dd'] >= tau) & (EDD['Job']).isin(Jc)].sort_values(by = 'Pj', ascending = False)
        j_star = L.iloc[0, 0]
        P_j_star = L.iloc[0, 1]
        sequence.insert(0, j_star)
        total_completion_time = SPT_EDD['Pj'].sum()  # Replace SPT_EDD with the sequence produced
        total_completion_times.append(total_completion_time)
        maximum_latenesses.append(L)
        # Step 4
        l_new = EDD.loc[EDD['dd'] < tau].sort_values(by = 'dd', ascending = False)
        if len(list(l_new['Job'])) != 0:
            delta_star_star = tau - l_new.iloc[0, 3]
            if delta_star_star < delta:
                delta = delta_star_star

        # Step 5
        k -= 1
        tau -= P_j_star
        Jc.remove(j_star)

    # Step 6
    r += 1
    EDD['dd'] += delta
    Lmax += delta
    print("Optimal Job Sequence: ", sequence)

Optimal Job Sequence:  [8, 4, 2, 1, 7, 5, 3, 9, 10, 6, 11]
